<a href="https://colab.research.google.com/github/shndap/Compiler-Design-HW1/blob/main/C_Minus_Scanner_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import re
TOKEN_TYPES = ['NUM', 'ID', 'KEYWORD', 'SYMBOL', 'ERROR']
KEYWORDS = sorted(['if', 'else', 'void', 'int', 'for', 'break', 'return'])
SINGLE_SYMBOLS = [';', ':', ',', '[', ']', '(', ')', '{', '}', '+', '-', '*', '<']
AMBIGUOUS_SYMBOLS = ['=', '/']
INPUT_FILE = "input.txt"
TOKENS_FILE = "tokens.txt"
ERRORS_FILE = "lexical_errors.txt"
SYMBOL_TABLE_FILE = "symbol_table.txt"

class Scanner:
    def __init__(self):
        """Initializes the scanner, loads the input, and sets up state variables."""
        try:
            with open(INPUT_FILE, 'r') as f:
                self.input_text = f.read()
        except FileNotFoundError:
            print(f"Error: Input file '{INPUT_FILE}' not found.")
            sys.exit(1)
        self.pointer = 0
        self.lineno = 1
        self.token_lines = {1: []}
        self.errors = []
        self.symbol_table = {'!'+k: k for i, k in enumerate(KEYWORDS, 1)}
        self.symbol_index_counter = len(KEYWORDS) + 1
        self.last_token_lineno = 1

    def _get_char(self):
        """Reads the current character and advances the pointer."""
        if self.pointer < len(self.input_text):
            char = self.input_text[self.pointer]
            self.pointer += 1
            return char
        return None

    def _peek_char(self):
        """Peeks at the next character without advancing the pointer."""
        if self.pointer < len(self.input_text):
            return self.input_text[self.pointer]
        return None

    def _unget_char(self):
        """Moves the pointer back one position (undoes a _get_char)."""
        if self.pointer > 0:
            self.pointer -= 1

    def _update_lineno(self, char):
        """Checks if the character is a newline and updates the line number."""
        if char == '\n':
            self.lineno += 1
            if self.lineno not in self.token_lines:
                self.token_lines[self.lineno] = []

    def _record_error(self, error_str, message):
        """Records a lexical error with the current line number."""
        self.errors.append((self.last_token_lineno, error_str, message))

    def _panic_mode_recovery(self, initial_char, error_str, message):
        """
        Implements Panic Mode error recovery: records the error, discards
        current token buffer, and discards subsequent characters until a valid
        token start is found.
        Returns ('EOF', '') if EOF is hit during recovery, else returns None
        (implicitly) indicating the scanner is ready to try finding a new token.
        """
        self._record_error(error_str, message)
        while True:
            char = self._get_char()
            if char is None:
                return ('EOF', '')

            self._update_lineno(char)
            if char.isalpha() or char == '_' or char.isdigit() or char in SINGLE_SYMBOLS or char in AMBIGUOUS_SYMBOLS:
                self._unget_char()
                return None
            if message == 'Illegal character' and error_str == initial_char:
                return None

    def _process_identifier_or_keyword(self, first_char):
        """
        DFA for IDs and KEYWORDS.
        Accepts: [A-Za-z_] [A-Za-z0-9_]*
        """
        lexeme = first_char
        while True:
            char = self._peek_char()
            if char is not None and (char.isalnum() or char == '_'):
                lexeme += self._get_char()
                self._update_lineno(lexeme[-1])
            else:
                break

        if lexeme in KEYWORDS:
            return ('KEYWORD', lexeme)
        else:
            if lexeme not in self.symbol_table:
                self.symbol_table[lexeme] = self.symbol_index_counter
                self.symbol_index_counter += 1
            return ('ID', lexeme)

    def _process_number(self, first_char):
        """
        DFA for NUMBERS.
        Accepts: [0-9]+
        Checks for: leading zeros (except '0') and malformed numbers (e.g., 125d).
        Returns ('NUM', lexeme) for valid numbers, ('EOF', '') if EOF hit during recovery,
        or None if panic mode recovery occurred and repositioned the pointer without EOF.
        """
        lexeme = first_char
        if first_char == '0':
            peek = self._peek_char()
            if peek is not None and peek.isdigit():
                lexeme += self._get_char()
                while True:
                    char = self._peek_char()
                    if char is not None and (char.isalnum() or char == '_'):
                        lexeme += self._get_char()
                    else:
                        break
                panic_result = self._panic_mode_recovery(first_char, lexeme, 'Malformed number')
                return panic_result
            elif peek is not None and peek.isalpha():
                lexeme += self._get_char()
                while True:
                    char = self._peek_char()
                    if char is not None and (char.isalnum() or char == '_'):
                        lexeme += self._get_char()
                    else:
                        break
                panic_result = self._panic_mode_recovery(first_char, lexeme, 'Malformed number')
                return panic_result
            else:
                return ('NUM', '0')
        while True:
            char = self._peek_char()
            if char is not None and char.isdigit():
                lexeme += self._get_char()
            elif char is not None and char.isalpha() or char == '_':
                lexeme += self._get_char()
                while True:
                    char = self._peek_char()
                    if char is not None and (char.isalnum() or char == '_'):
                        lexeme += self._get_char()
                    else:
                        break
                panic_result = self._panic_mode_recovery(first_char, lexeme, 'Malformed number')
                return panic_result
            else:
                return ('NUM', lexeme)

    def _skip_whitespace_and_comments(self):
        """
        DFA for WHITESPACE and COMMENTS.
        Handles / vs // vs /* ... */ and EOF error for block comment.
        Returns the first non-skipped character or None (EOF).
        """
        while True:
            char = self._get_char()
            if char is None:
                return None
            if char.isspace() and char != '/':
                self._update_lineno(char)
                continue
            elif char == '/':
                peek = self._peek_char()

                if peek == '/':
                    self._get_char()
                    while True:
                        comment_char = self._get_char()
                        if comment_char is None or comment_char in ['\n', '\f']:
                            if comment_char == '\n':
                                self._update_lineno(comment_char)
                            break
                    continue

                elif peek == '*':
                    self._get_char()
                    comment_lexeme = "/*"
                    comment_lineno = self.lineno

                    while True:
                        comment_char = self._get_char()
                        if comment_char is None:
                            error_str = comment_lexeme[:10] + '...' if len(comment_lexeme) > 10 else comment_lexeme
                            self._record_error(error_str, 'Open comment at EOF')
                            return ('EOF', '')

                        self._update_lineno(comment_char)
                        comment_lexeme += comment_char

                        if comment_char == '*':
                            if self._peek_char() == '/':
                                self._get_char()
                                break
                    continue

                else:
                    return char
            elif char == '*':
                if self._peek_char() == '/':
                    self._get_char()
                    self._record_error('*/', 'Stray closing comment')
                    continue
                else:
                    return char
            else:
                return char

    def get_next_token(self):
        """
        The main DFA-driven function to recognize the next token.
        Uses single lookahead logic for ambiguous symbols.
        """
        while True:
            char_or_token = self._skip_whitespace_and_comments()
            if char_or_token == ('EOF', ''):
                return char_or_token
            if char_or_token is None:
                return ('EOF', '')

            char = char_or_token
            self.last_token_lineno = self.lineno
            if char.isalpha() or char == '_':
                return self._process_identifier_or_keyword(char)
            elif char.isdigit():
                token = self._process_number(char)
                if token is not None:
                    return token
                continue
            elif char in SINGLE_SYMBOLS:
                return ('SYMBOL', char)
            elif char in AMBIGUOUS_SYMBOLS:
                peek = self._peek_char()

                if char == '=':
                    if peek == '=':
                        self._get_char()
                        return ('SYMBOL', '==')
                    else:
                        return ('SYMBOL', '=')

                elif char == '/':
                    return ('SYMBOL', '/')
            error_str = char
            panic_result = self._panic_mode_recovery(char, error_str, 'Illegal character')
            if panic_result == ('EOF', ''):
                return panic_result
            continue

    def run(self):
        """
        The main execution loop to scan the entire input and generate output files.
        """
        while True:
            token = self.get_next_token()
            token_type, token_str = token

            if token_type == 'EOF':
                break

            if token_type not in ['EOF', 'ERROR']:
                self.token_lines[self.last_token_lineno].append(f"({token_type}, {token_str})")
        with open(TOKENS_FILE, 'w') as f:
            for line_num in sorted(self.token_lines.keys()):
                tokens_on_line = self.token_lines[line_num]
                if tokens_on_line:
                    f.write(f"{line_num}\t{' '.join(tokens_on_line)}\n")
        with open(ERRORS_FILE, 'w') as f:
            if not self.errors:
                f.write('No lexical errors found.\n')
            else:
                for line_num, error_str, message in self.errors:
                    f.write(f"{line_num}\t({error_str}, {message})\n")
        sorted_symbols = sorted([(k, v) for k, v in self.symbol_table.items()], key=lambda item: item[0])
        final_symbol_table = {lexeme: i + 1 for i, (lexeme, _) in enumerate(sorted_symbols)}

        with open(SYMBOL_TABLE_FILE, 'w') as f:
            f.write("no\tlexeme\n")
            for i, (lexeme, _) in enumerate(sorted_symbols, 1):
                f.write(f"{i}\t{lexeme[lexeme[0]=='!':]}\n")
scanner = Scanner()
scanner.run()

In [ ]:
s = """void main (void) {
int a_var = 0;
/* comment1*/
a_var = 2 + 2;
a_var = a_var - 3;
cde = a_var;
if (b /* comment2 */ == 3d) {
a_var = 3;
cd!e = 7;
}
else */
{
b = a_var < cde;
{cde = @2;
}}
return; // comment 3
/* open comment
}
"""
with open("input.txt", "w") as f:
    f.write(s)


In [ ]:
def pf(z):
  print(f"==========| {z} |==========")
  with open(f"{z}.txt", "r") as f:
    for l in f:
      print(l, end='')

pf('input')
pf('lexical_errors')
pf('symbol_table')
pf('tokens')

==========| input |==========
void main (void) {
int a_var = 0;
/* comment1*/
a_var = 2 + 2;
a_var = a_var - 3;
cde = a_var;
if (b /* comment2 */ == 3d) {
a_var = 3;
cd!e = 7;
}
else */
{
b = a_var < cde;
{cde = @2;
}}
return; // comment 3
/* open comment
}
==========| lexical_errors |==========
7	(3d, Malformed number)
9	(!, Illegal character)
11	(*/, Stray closing comment)
14	(@, Illegal character)
16	(/* open co..., Open comment at EOF)
==========| symbol_table |==========
no	lexeme
1	break
2	else
3	for
4	if
5	int
6	return
7	void
8	a_var
9	b
10	cd
11	cde
12	e
13	main
==========| tokens |==========
1	(KEYWORD, void) (ID, main) (SYMBOL, () (KEYWORD, void) (SYMBOL, )) (SYMBOL, {)
2	(KEYWORD, int) (ID, a_var) (SYMBOL, =) (NUM, 0) (SYMBOL, ;)
4	(ID, a_var) (SYMBOL, =) (NUM, 2) (SYMBOL, +) (NUM, 2) (SYMBOL, ;)
5	(ID, a_var) (SYMBOL, =) (ID, a_var) (SYMBOL, -) (NUM, 3) (SYMBOL, ;)
6	(ID, cde) (SYMBOL, =) (ID, a_var) (SYMBOL, ;)
7	(KEYWORD, if) (SYMBOL, () (ID, b) (SYMBOL, ==) (SYMBOL, )) (